## 💾데이터 전처리 실습_1: 
**❗목표:** 7년 방문자 데이터를 범용적으로 분석 가능한 Long 형태로 변환

##### [원본: 03_중앙박물관_7개년_관람객](data/raw/museum/csv/03_중앙박물관_7개년_관람객.csv)
##### [정제: museum_visitors_7years_demographics ](../data/processed/museum_visitors_7years_demographics.csv)

In [6]:
import pandas as pd

# 1) CSV 파일 읽기
df = pd.read_csv('../data/raw/museum/csv/03_중앙박물관_7개년_관람객.csv')

# 2) 행열 전환
df = df.transpose()

# 3) 첫 번째 행을 컬럼명으로 지정
df.columns = df.iloc[0]

# 4) 첫 번째 행 제거 + 인덱스를 컬럼으로 변환
df = df[1:].reset_index()

display(df)


구분,index,전체 관람객(명),외국인(명),비율(%)
0,2019년,3354161.0,138464.0,4.13
1,2020년,773621.0,17593.0,2.27
2,2021년,1262562.0,13996.0,1.11
3,2022년,3411381.0,70279.0,2.06
4,2023년,4180285.0,172077.0,4.12
5,2024년,3788785.0,198085.0,5.23
6,2025년,6507483.0,231192.0,3.55


In [7]:
# 현재 데이터 타입 확인
print("데이터 타입 확인:")
print(df.dtypes)

# 샘플 데이터 확인
print("\n샘플 데이터:")
display(df.head(3))

데이터 타입 확인:
구분
index        object
전체 관람객(명)    object
외국인(명)       object
비율(%)        object
dtype: object

샘플 데이터:


구분,index,전체 관람객(명),외국인(명),비율(%)
0,2019년,3354161.0,138464.0,4.13
1,2020년,773621.0,17593.0,2.27
2,2021년,1262562.0,13996.0,1.11


In [8]:
# 비율 컬럼 삭제 
df = df.drop(columns='비율(%)', errors='ignore')

# 숫자 컬럼을 numeric으로 변환 (핵심!)
df['전체 관람객(명)'] = pd.to_numeric(df['전체 관람객(명)'])
df['외국인(명)'] = pd.to_numeric(df['외국인(명)'])

# korean 컬럼 추가
df['korean'] = df['전체 관람객(명)'] - df['외국인(명)']

# korean_ratio 컬럼 추가 (퍼센트로, 소수점 2자리)
df['korean_ratio'] = ((df['korean'] / df['전체 관람객(명)'])).round(4)

# foreigner_ratio 컬럼 추가 (퍼센트로, 소수점 2자리)
df['foreigner_ratio'] = ((df['외국인(명)'] / df['전체 관람객(명)'])).round(4)

# 결과 확인
display(df)
print(f"\n컬럼 목록: {df.columns.tolist()}")

구분,index,전체 관람객(명),외국인(명),korean,korean_ratio,foreigner_ratio
0,2019년,3354161.0,138464.0,3215697.0,0.9587,0.0413
1,2020년,773621.0,17593.0,756028.0,0.9773,0.0227
2,2021년,1262562.0,13996.0,1248566.0,0.9889,0.0111
3,2022년,3411381.0,70279.0,3341102.0,0.9794,0.0206
4,2023년,4180285.0,172077.0,4008208.0,0.9588,0.0412
5,2024년,3788785.0,198085.0,3590700.0,0.9477,0.0523
6,2025년,6507483.0,231192.0,6276291.0,0.9645,0.0355



컬럼 목록: ['index', '전체 관람객(명)', '외국인(명)', 'korean', 'korean_ratio', 'foreigner_ratio']


In [9]:
df['index']= df['index'].str.replace('년', '').astype(int)
df['index']

0    2019
1    2020
2    2021
3    2022
4    2023
5    2024
6    2025
Name: index, dtype: int64

In [10]:
# 5) 컬럼명 정리
df = df.rename(columns={'index': 'year'})  # index → year
df.columns.name = None                      # 컬럼 헤더 위 라벨 제거

# 6) year를 인덱스로 설정
df = df.set_index('year')

df = df.rename(columns={'전체 관람객(명)': 'total_visitors','외국인(명)':'foreigner'}) 
# df[['total_visitor','foreigner']].astype(int)
display(df)

,total_visitors,foreigner,korean,korean_ratio,foreigner_ratio
year,,,,,
2019,3354161.0,138464.0,3215697.0,0.9587,0.0413
2020,773621.0,17593.0,756028.0,0.9773,0.0227
2021,1262562.0,13996.0,1248566.0,0.9889,0.0111
2022,3411381.0,70279.0,3341102.0,0.9794,0.0206
2023,4180285.0,172077.0,4008208.0,0.9588,0.0412
2024,3788785.0,198085.0,3590700.0,0.9477,0.0523
2025,6507483.0,231192.0,6276291.0,0.9645,0.0355


In [11]:
# 현재 데이터 타입 확인
print("데이터 타입 확인:")
print(df.dtypes)

데이터 타입 확인:
total_visitors     float64
foreigner          float64
korean             float64
korean_ratio       float64
foreigner_ratio    float64
dtype: object


In [12]:
# 컬럼 순서 정리
df= df[['korean', 'foreigner', 'total_visitors', 'korean_ratio', 'foreigner_ratio']]
display(df)

,korean,foreigner,total_visitors,korean_ratio,foreigner_ratio
year,,,,,
2019,3215697.0,138464.0,3354161.0,0.9587,0.0413
2020,756028.0,17593.0,773621.0,0.9773,0.0227
2021,1248566.0,13996.0,1262562.0,0.9889,0.0111
2022,3341102.0,70279.0,3411381.0,0.9794,0.0206
2023,4008208.0,172077.0,4180285.0,0.9588,0.0412
2024,3590700.0,198085.0,3788785.0,0.9477,0.0523
2025,6276291.0,231192.0,6507483.0,0.9645,0.0355


In [13]:
# 최종 결과 확인
print(f"\n1. 최종 shape:\n{df.shape}")
print(f"\n2. 컬럼 목록:\n{df.columns.tolist()}")
print(f"\n3. 데이터 타입 확인:\n{df.dtypes}")
# 기본 통계
print("\n4. 기본 통계:")
display(df.describe())

# 데이터 미리보기
print("\n5. 데이터 5개 미리보기:")
display(df.style.format({'total_visitors': '{:,}', 'korean': '{:,}','foreigner': '{:,}'}))
# display(df.head(5))


1. 최종 shape:
(7, 5)

2. 컬럼 목록:
['korean', 'foreigner', 'total_visitors', 'korean_ratio', 'foreigner_ratio']

3. 데이터 타입 확인:
korean             float64
foreigner          float64
total_visitors     float64
korean_ratio       float64
foreigner_ratio    float64
dtype: object

4. 기본 통계:


,korean,foreigner,total_visitors,korean_ratio,foreigner_ratio
count,7.000000e+00,7.000000,7.000000e+00,7.000000,7.000000
mean,3.205227e+06,120240.857143,3.325468e+06,0.967900,0.032100
std,1.828956e+06,87282.753866,1.908154e+06,0.014426,0.014426
min,7.560280e+05,13996.000000,7.736210e+05,0.947700,0.011100
25%,2.232132e+06,43936.000000,2.308362e+06,0.958750,0.021650
50%,3.341102e+06,138464.000000,3.411381e+06,0.964500,0.035500
75%,3.799454e+06,185081.000000,3.984535e+06,0.978350,0.041250
max,6.276291e+06,231192.000000,6.507483e+06,0.988900,0.052300



5. 데이터 5개 미리보기:


,korean,foreigner,total_visitors,korean_ratio,foreigner_ratio
year,,,,,
2019,"3,215,697.0","138,464.0","3,354,161.0",0.958700,0.041300
2020,"756,028.0","17,593.0","773,621.0",0.977300,0.022700
2021,"1,248,566.0","13,996.0","1,262,562.0",0.988900,0.011100
2022,"3,341,102.0","70,279.0","3,411,381.0",0.979400,0.020600
2023,"4,008,208.0","172,077.0","4,180,285.0",0.958800,0.041200
2024,"3,590,700.0","198,085.0","3,788,785.0",0.947700,0.052300
2025,"6,276,291.0","231,192.0","6,507,483.0",0.964500,0.035500


In [14]:
# 7) CSV 저장 (인덱스 포함)
df.to_csv('../data/processed/museum_visitors_7years_demographics.csv', 
          
          encoding='utf-8-sig', 
          index=True)

In [15]:
display(df)

,korean,foreigner,total_visitors,korean_ratio,foreigner_ratio
year,,,,,
2019,3215697.0,138464.0,3354161.0,0.9587,0.0413
2020,756028.0,17593.0,773621.0,0.9773,0.0227
2021,1248566.0,13996.0,1262562.0,0.9889,0.0111
2022,3341102.0,70279.0,3411381.0,0.9794,0.0206
2023,4008208.0,172077.0,4180285.0,0.9588,0.0412
2024,3590700.0,198085.0,3788785.0,0.9477,0.0523
2025,6276291.0,231192.0,6507483.0,0.9645,0.0355
